In [ ]:
import json
import os
import requests
from pydantic import BaseModel
from typing import List
from groq import Groq
import os
from dotenv import load_dotenv
import time
import requests 

load_dotenv()


# Define our Pydantic models to structure the parsed output.
class TweetObject(BaseModel):
    Author: str         # e.g. @elonmusk
    Time: str           # e.g. 2h ago
    Content: str        # e.g. Tweet's textual content
    Likes: float        # e.g. 109000 (converted from "109K")
    Comments: float     # e.g. 3400 (converted from "3.4K")
    Reposts: float      # e.g. 19000 (converted from "19K")
    Views: float        # e.g. 22000000 (converted from "22M")

class TweetList(BaseModel):
    tweets: List[TweetObject]


def perform_web_action(session_id: str, task: str, url: str = "https://x.com/home", verify: bool = True) -> dict:
    """
    Enhanced helper function with verification and better error handling
    """
    url_task = "https://connect.anchorbrowser.io/tools/perform-web-task"
    querystring = {
        "apiKey": os.getenv("ANCHOR_API_KE"),
        "sessionId": session_id
    }
    
    # First verify we're on the correct page if verification is requested
    if verify:
        verify_payload = {
            "task": "Wait for page to load completely and verify current URL",
            "url": url,
            "wait": 2000
        }
        verify_response = requests.post(
            url_task,
            json=verify_payload,
            headers={"Content-Type": "application/json"},
            params=querystring
        )
        time.sleep(2)  # Wait after verification
    
    # Perform the actual task
    payload_task = {
        "task": task,
        "url": url,
        "wait": 8,  # 8 second wait
        "timeout": 2000  # 20 second timeout
    }
    
    try:
        response = requests.post(
            url_task,
            json=payload_task,
            headers={"Content-Type": "application/json"},
            params=querystring,
            timeout=25
        )
        
        if response.status_code == 504:
            print(f"Timeout occurred. Retrying with longer wait...")
            payload_task["wait"] = 15000  # Increase wait time
            response = requests.post(
                url_task,
                json=payload_task,
                headers={"Content-Type": "application/json"},
                params=querystring,
                timeout=30
            )
        
        return {
            "status": response.status_code,
            "text": response.text,
            "success": response.status_code == 200
        }
        
    except Exception as e:
        return {"success": False, "error": str(e)}

def get_web_task_result() -> str:
    """
    Broken down into smaller, more manageable steps with verification
    """
    # Initialize session
    url_session = "https://api.anchorbrowser.io/api/sessions"
    payload_session = {"profile": {"name": "YOUR_SESSION_NAME"}}
    headers_session = {
        "anchor-api-key": os.getenv("ANCHOR_API_KEY"),
        "Content-Type": "application/json"
    }

    response_session = requests.post(url_session, json=payload_session, headers=headers_session)
    session_id = response_session.json()["id"]
    
    # Break down the process into smaller steps
    steps = [
        {
            "task": "Click on the heart icon of the five most recent posts you see to like them.",
            "url": "https://x.com/home",
            "verify": True,
            "description": "Liking posts"
        }
    ]
    
    results = []
    for step in steps:
        print(f"\nExecuting: {step['description']}...")
        
        # Try the action up to 3 times
        for attempt in range(3):
            result = perform_web_action(
                session_id,
                step['task'],
                step['url'],
                step.get('verify', True)
            )
            
            if result.get("success"):
                results.append(f"{step['description']}: Success")
                break
            elif attempt < 2:  # Only print retry message if we're going to retry
                print(f"Attempt {attempt + 1} failed, retrying after delay...")
                time.sleep(5)  # 5 second delay between retries
            else:
                results.append(f"{step['description']}: Failed after 3 attempts")
        
        # Always wait between steps
        time.sleep(3)
    
    return "\n".join(results)

if __name__ == '__main__':    
    try:
        result = get_web_task_result()
        print("\nFinal Results:")
        print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        print(traceback.format_exc())